# 🧪 Evaluate Models in `results/`

这个 Notebook 用于评估/可视化你在 `results/` 下训练出的模型：
- **手动指定模型路径**：直接填 `MODEL_PATH`（绝对/相对路径都可）。
- **按算法自动加载最新模型**：只选 `ALGO_TAG`，脚本会自动寻找该算法的最新 `best_model*.pt`。

自动搜索会兼容三种位置：
1) 新结构：`results/<algo>/**/models/`
2) 旧结构：`results/**/models/`（会跳过 `results/eval/`）
3) 更旧：项目根目录下的 `best_model*.pt`

评估产物（指标 JSON、图像）**默认不保存**；如需落盘，可在第 3 步把 `SAVE_OUTPUTS=True`，保存到：`results/eval/<algo>/YYYYMMDD/HHMMSS/`（与训练产物分开）。


In [ ]:
import os
import json
from pathlib import Path
from datetime import datetime

import torch

# 项目模块
from config import (
    DEVICE, SEED, set_seed,
    NUM_FOLLOWERS, NUM_PINNED, MAX_STEPS,
)
from topology import CommunicationTopology
from environment import ModelFreeEnv
from agent import CTDESACAgent, CTDEMAPPOAgent
from utils import evaluate_agent, plot_evaluation

print('DEVICE:', DEVICE)
print('torch:', torch.__version__)


In [ ]:
def _algo_tag_normalize(tag: str) -> str:
    tag = (tag or '').strip().lower()
    if tag in ('masac', 'sac', 'ctdesac', 'ctde_sac'):
        return 'masac'
    if tag in ('mappo', 'ppo', 'ctdemappo', 'ctde_mappo'):
        return 'mappo'
    return tag or 'masac'


def find_latest_model_in_results(algo_tag: str, results_root: str | Path = 'results', pattern: str = 'best_model*.pt') -> str:
    """按修改时间找“最新模型”。搜索顺序：
    1) results/<algo>/**/models/<pattern>
    2) 兼容旧结构：results/**/models/<pattern>（自动跳过 results/eval）
    3) 兼容更旧：项目根目录 ./best_model*.pt
    """
    algo_tag = _algo_tag_normalize(algo_tag)
    results_root = Path(results_root)

    def _is_algo_match(p: Path) -> bool:
        n = p.name.lower()
        if algo_tag == 'mappo':
            return ('mappo' in n) or ('ppo' in n and 'masac' not in n)
        if algo_tag == 'masac':
            # 允许 generic best_model.pt；同时排除 mappo/ppo
            return ('masac' in n) or (('mappo' not in n) and ('ppo' not in n))
        return True

    def _collect_from_dir(base: Path) -> list[Path]:
        out: list[Path] = []
        if not base.exists():
            return out
        for p in base.rglob(pattern):
            if not (p.is_file() and p.parent.name == 'models'):
                continue
            # 跳过 eval 目录
            if 'eval' in p.parts:
                continue
            if _is_algo_match(p):
                out.append(p)
        return out

    candidates: list[Path] = []

    # 1) 新结构：results/<algo>/...
    candidates += _collect_from_dir(results_root / algo_tag)

    # 2) 旧结构：results/YYYYMMDD/HHMMSS/models/...
    if not candidates:
        candidates += _collect_from_dir(results_root)

    # 3) 更旧：项目根目录
    if not candidates:
        for p in Path('.').glob(pattern):
            if p.is_file() and _is_algo_match(p):
                candidates.append(p)

    if not candidates:
        raise FileNotFoundError(
            f"未找到模型：algo={algo_tag}, pattern={pattern}。\n"
            f"已搜索：{results_root / algo_tag}、{results_root}（跳过 eval）、以及项目根目录。"
        )

    latest = max(candidates, key=lambda x: x.stat().st_mtime)
    return str(latest)


def guess_algo_from_model_path(model_path: str) -> str:
    name = Path(model_path).name.lower()
    if 'mappo' in name or 'ppo' in name:
        return 'mappo'
    return 'masac'


def make_eval_output_dirs(algo_tag: str, results_root: str | Path = 'results') -> dict:
    algo_tag = _algo_tag_normalize(algo_tag)
    now = datetime.now()
    date = now.strftime('%Y%m%d')
    t = now.strftime('%H%M%S')

    base = Path(results_root) / 'eval' / algo_tag / date / t
    models_dir = base / 'models'
    figs_dir = base / 'figs'
    base.mkdir(parents=True, exist_ok=True)
    models_dir.mkdir(parents=True, exist_ok=True)
    figs_dir.mkdir(parents=True, exist_ok=True)

    return {
        'base': base,
        'models_dir': models_dir,
        'figs_dir': figs_dir,
    }


## 1) 选择模型

你可以二选一：
- 方式 A：手动指定 `MODEL_PATH`
- 方式 B：只指定 `ALGO_TAG`，自动加载该算法最新模型


In [ ]:
# ========== 方式 A（手动）==========
MODEL_PATH = ''  # e.g. '/root/.../results/masac/20251228/231613/models/best_model_ctde_14f_masac_light.pt'

# ========== 方式 B（自动）==========
ALGO_TAG = 'masac'  # 'masac' or 'mappo'
AUTO_LOAD_LATEST = True

# 模型文件名匹配规则（一般不用改）
MODEL_GLOB = 'best_model*.pt'

# -------- 决策逻辑 --------
if MODEL_PATH:
    model_path = MODEL_PATH
    algo_tag = guess_algo_from_model_path(model_path)
else:
    algo_tag = _algo_tag_normalize(ALGO_TAG)
    if AUTO_LOAD_LATEST:
        model_path = find_latest_model_in_results(algo_tag, results_root='results', pattern=MODEL_GLOB)
    else:
        raise ValueError('AUTO_LOAD_LATEST=False 时请手动填写 MODEL_PATH')

model_path = str(Path(model_path).expanduser())
print('Selected algo_tag:', algo_tag)
print('Selected model_path:', model_path)
assert Path(model_path).exists(), f'模型文件不存在: {model_path}'


## 2) 加载模型并评估（输出数值指标）

默认会评估 `num_eval_episodes` 个 episode，输出平均 reward / 平均跟踪误差 / 平均通信率。


In [ ]:
# 评估参数
num_eval_episodes = 10
seed_for_eval = SEED

# 画图时最多展示多少个跟随者：None=全部；例如 5=只画前 5 个
MAX_PLOT_FOLLOWERS = None

set_seed(seed_for_eval)

# 环境/拓扑
topology = CommunicationTopology(NUM_FOLLOWERS, num_pinned=NUM_PINNED)
env = ModelFreeEnv(topology)

# agent
if algo_tag == 'mappo':
    agent = CTDEMAPPOAgent(topology, use_amp=False)
else:
    agent = CTDESACAgent(topology, use_amp=False)

agent.load(model_path)

metrics = evaluate_agent(agent, env, num_episodes=num_eval_episodes)
print('Metrics:', metrics)


## 3) （可选）保存评估产物（指标 JSON + 评估图）

默认不保存任何生成物；把下面单元格里的 `SAVE_OUTPUTS=True` 后再运行即可保存到 `results/eval/<algo>/YYYYMMDD/HHMMSS/`。


In [ ]:
# 默认不保存任何生成物；需要落盘时改成 True 并重新运行本单元格
SAVE_OUTPUTS = False

if SAVE_OUTPUTS:
    out = make_eval_output_dirs(algo_tag, results_root='results')
    print('Eval outputs ->', out['base'])

    # 1) 保存指标
    metrics_path = out['base'] / 'metrics.json'
    payload = {
        'algo': algo_tag,
        'model_path': model_path,
        'seed': seed_for_eval,
        'num_eval_episodes': num_eval_episodes,
        'metrics': metrics,
        'timestamp': datetime.now().isoformat(timespec='seconds'),
    }
    metrics_path.write_text(json.dumps(payload, indent=2, ensure_ascii=False), encoding='utf-8')
    print('Saved metrics ->', metrics_path)

    # 2) 保存评估图
    fig_path = out['figs_dir'] / f'final_evaluation_{algo_tag}.png'
    plot_evaluation(agent, topology, num_tests=3, save_path=str(fig_path), max_plot_followers=MAX_PLOT_FOLLOWERS)
    print('Saved figure ->', fig_path)
else:
    print('SAVE_OUTPUTS=False：本次仅评估并打印 metrics，不保存 metrics.json/图片。')
    # 仍可在 Notebook 里直接看图（不落盘）
    plot_evaluation(agent, topology, num_tests=3, save_path=None, max_plot_followers=MAX_PLOT_FOLLOWERS)


## 可选：固定环境随机化（做可复现对比）

你的环境默认 `reset()` 会随机化 leader 参数/初始状态/拓扑。
如果你想在**同一个固定场景**里对比多个模型，可以把随机化关掉。


In [ ]:
# 关闭随机化（可选）
env_fixed = ModelFreeEnv(topology)
env_fixed.batched_env.randomize_leader = False
env_fixed.batched_env.randomize_follower = False
env_fixed.batched_env.randomize_topology = False

metrics_fixed = evaluate_agent(agent, env_fixed, num_episodes=5)
print('Fixed-scenario metrics:', metrics_fixed)
